In [50]:
import gurobipy as gp
import numpy as np
import pandas as pd
from math import *
from copy import deepcopy

### Ler capacidades

In [51]:
df_oferta = pd.read_excel('../dados/ibge_prod.xlsx', index_col=0)
df_oferta.head()

,2017,2018,2019,2020,2021,Média
"Brasil, Unidade da Federação e Município",,,,,,
Goiás,11372539,11476946,11080442,12849800,13654117,12086768.8
"Rio Verde, GO, Brazil",1102500,1170000,1081200,1404000,1476000,1246740.0
"Jataí, GO, Brazil",1060200,1026000,1061900,1060200,1078800,1057420.0
"Cristalina, GO, Brazil",811200,860000,694100,936000,993750,859010.0
"Montividiu, GO, Brazil",455880,454860,421200,496800,468600,459468.0


In [52]:
oferta_total = df_oferta['Média'].sum() - df_oferta['Média']['Goiás']
N = [i for i in (set(df_oferta.index) - set(['Goiás']))]
qnt_orig = len(N)

In [53]:
supply = {}
for i in N:
    supply[i.lower()] = df_oferta['Média'][i]
supply

{"são joão d'aliança, go, brazil": 124600.0,
 'montes claros de goiás, go, brazil': 116013.6,
 'caldas novas, go, brazil': 106740.0,
 'niquelândia, go, brazil': 164300.0,
 'quirinópolis, go, brazil': 77460.0,
 'ipameri, go, brazil': 333080.0,
 'uruaçu, go, brazil': 78360.0,
 'rio verde, go, brazil': 1246740.0,
 'doverlândia, go, brazil': 61636.0,
 'goiatuba, go, brazil': 257122.0,
 'gameleira de goiás, go, brazil': 92008.0,
 'bom jesus de goiás, go, brazil': 201246.0,
 'santa cruz de goiás, go, brazil': 66688.0,
 'luziânia, go, brazil': 217160.0,
 'cabeceiras, go, brazil': 98780.0,
 'turvelândia, go, brazil': 84780.0,
 'chapadão do céu, go, brazil': 320980.0,
 'silvânia, go, brazil': 261880.0,
 'serranópolis, go, brazil': 106048.0,
 'catalão, go, brazil': 405080.0,
 'campo alegre de goiás, go, brazil': 237390.0,
 'paraúna, go, brazil': 395796.0,
 'vicentinópolis, go, brazil': 73612.0,
 'portelândia, go, brazil': 80490.0,
 'padre bernardo, go, brazil': 186256.0,
 'cristalina, go, brazil

In [54]:
df_cap_porto = pd.read_excel('../dados/ibge_cap_porto.xlsx')
df_cap_porto.head()

,"SANTOS, SP, Brazil","VITÓRIA, ES, Brazil","SÃO LUIZ, MA, Brazil","ILHÉUS, BA, Brazil","SALVADOR, BA, Brazil","RIO GRANDE, RS, Brazil","SÃO FRANCISCO DO SUL, SC, Brazil",Total
0,5922516.712,725206.128,1933883.008,725206.128,846073.816,145041225.6,483470.752,1.556776e+08


In [55]:
demanda_total = df_cap_porto['Total'][0]
M = [i for i in (set(df_cap_porto.columns) - set(['Total']))]
qnt_port = len(M)

In [56]:
demanda_total

155677582.144

In [57]:
cap_port = {}
for i in M:
    cap_port[i.lower()] = df_cap_porto[i][0]
print(sum([cap_port[i] for i in cap_port]))

155677582.144


In [58]:
df_cap_trans = pd.read_excel('../dados/ibge_cap_transbordo.xlsx')
df_cap_trans.head()

,"PORTO NACIONAL, TO, Brazil","FIGUEIRÓPOLIS, TO, Brazil","PORANGATU, GO, Brazil","ANÁPOLIS, GO, Brazil","IPAMERI,GO, Brazil","RIO VERDE, GO, Brazil","SÃO SIMÃO, GO, Brazil"
0,374689.8328,4.278716e+06,1.136156e+06,459297.2144,277995.6824,4411670.612,1148243.036


In [59]:
K = [i for i in (set(df_cap_trans.columns))]
qnt_trans = len(K)

In [60]:
cap_trans = {}
for i in K:
    cap_trans[i.lower()] = df_cap_trans[i][0]
cap_trans

{'são simão, go, brazil': 1148243.036,
 'ipameri,go, brazil': 277995.6824,
 'porto nacional, to, brazil': 374689.8328,
 'anápolis, go, brazil': 459297.2144,
 'figueirópolis, to, brazil': 4278716.1552,
 'rio verde, go, brazil': 4411670.612,
 'porangatu, go, brazil': 1136156.2672}

In [61]:
O = [qnt_orig + qnt_trans + qnt_port]
demand = { O[0]: demanda_total }

### Ler matriz de frete

In [62]:
df_cost_origem_trans = pd.read_excel('../dados/ibge_cost_origem_transbordo.xlsx', index_col=0)
df_cost_origem_trans.head()

,"CANAÃ DOS CARAJÁS, PA, Brazil","PORTO NACIONAL, TO, Brazil","FIGUEIRÓPOLIS, TO, Brazil","PORANGATU, GO, Brazil","ANÁPOLIS, GO, Brazil","IPAMERI,GO, Brazil","RIO VERDE, GO, Brazil","SÃO SIMÃO, GO, Brazil","SÃO JOSÉ DA LAPA, MG, Brazil","LONDRINA, PR, Brazil","MARINGÁ, PR, Brazil"
Origem,,,,,,,,,,,
"Rio Verde, GO, Brazil",262.335670,167.546841,136.900218,110.254099,59.230208,70.679276,15.269992,41.576239,149.239214,131.288603,131.898136
"Jataí, GO, Brazil",254.072456,181.260231,150.613609,123.717505,72.693614,84.211998,29.054190,44.284930,162.771936,131.830609,132.440292
"Cristalina, GO, Brazil",250.655251,146.483120,116.543968,94.047270,49.687555,40.688249,89.256266,97.003262,103.129754,168.992788,166.818352
"Montividiu, GO, Brazil",261.273868,166.485038,135.838416,108.694861,65.845043,77.845359,22.668769,49.661021,156.405297,139.373384,139.982918
"Paraúna, GO, Brazil",243.427454,148.638475,117.991853,90.848298,47.998480,62.256710,34.204593,55.712980,157.585757,144.799412,152.264076


In [63]:
df_cost_origem_porto = pd.read_excel('../dados/ibge_cost_origem_porto.xlsx', index_col=0)
df_cost_origem_porto.head()

,"SANTOS, SP, Brazil","VITÓRIA, ES, Brazil","SÃO LUIZ, MA, Brazil","ILHÉUS, BA, Brazil","SALVADOR, BA, Brazil","RIO GRANDE, RS, Brazil","SÃO FRANCISCO DO SUL, SC, Brazil","BARCARENA, PA, Brazil","SANTAREM, PA, Brazil","ITACOATIARA, AM, Brazil","PARANAGUA, PR, Brazil","IMBITUBA, SC, Brazil"
Origem,,,,,,,,,,,,
"Rio Verde, GO, Brazil",164.863193,221.511668,352.994607,275.310584,294.872458,332.541761,214.890722,335.725379,383.039745,509.475583,203.497852,244.535318
"Jataí, GO, Brazil",173.330927,235.044539,366.707997,288.774139,308.336013,333.083768,215.432728,349.438770,369.915764,496.351602,204.039859,245.077324
"Cristalina, GO, Brazil",157.418802,181.280206,331.517971,206.704128,245.011288,344.596785,226.945745,323.571077,387.849680,574.595668,212.734025,256.590341
"Montividiu, GO, Brazil",172.029127,228.677751,351.932804,281.925419,301.487293,340.626394,222.975503,334.663726,375.880369,509.438763,211.582634,252.620099
"Paraúna, GO, Brazil",173.209586,229.858211,334.086390,264.078856,283.640879,360.387569,242.736679,316.817163,378.903740,526.603048,228.524809,272.381275


In [64]:
df_cost_trans_port = pd.read_excel('../dados/ibge_cost_transbordo_porto.xlsx', index_col=0)
df_cost_trans_port.head()

,"SANTOS, SP, Brazil","VITÓRIA, ES, Brazil","SÃO LUIZ, MA, Brazil","ILHÉUS, BA, Brazil","SALVADOR, BA, Brazil","RIO GRANDE, RS, Brazil","SÃO FRANCISCO DO SUL, SC, Brazil","BARCARENA, PA, Brazil","SANTAREM, PA, Brazil","ITACOATIARA, AM, Brazil","PARANAGUA, PR, Brazil","IMBITUBA, SC, Brazil"
"CANAÃ DOS CARAJÁS, PA, Brazil",1000000.00,1000000.00,92.36,1000000.00,1000000.0,1000000.0,1000000.0,1000000,1000000,1000000,1000000,1000000
"PORTO NACIONAL, TO, Brazil",170.61,170.61,116.93,170.61,1000000.0,1000000.0,1000000.0,1000000,1000000,1000000,1000000,1000000
"FIGUEIRÓPOLIS, TO, Brazil",170.61,170.61,170.61,170.61,1000000.0,1000000.0,1000000.0,1000000,1000000,1000000,1000000,1000000
"PORANGATU, GO, Brazil",170.61,170.61,170.61,170.61,1000000.0,1000000.0,1000000.0,1000000,1000000,1000000,1000000,1000000
"ANÁPOLIS, GO, Brazil",170.61,170.61,170.61,170.61,1000000.0,1000000.0,1000000.0,1000000,1000000,1000000,1000000,1000000


In [65]:
cost = {}
for n in N:
    for m in M:
        assert((n, m) not in cost)
        cost[n, m] = df_cost_origem_porto[m][n]
    for k in K:
        assert((n, k) not in cost)
        cost[n, k] = df_cost_origem_trans[k][n]
for k in K:
    for m in set(df_cost_trans_port.columns):
        assert((k, m) not in cost)
        cost[k, m] = df_cost_trans_port[m][k]
for j in M:
    cost[j, O[0]] = 0

## Ler matriz de CO2

In [66]:
df_emission_origem_trans = pd.read_excel('../dados/ibge_emission_origem_transbordo.xlsx', index_col=0)
df_emission_origem_trans.head()

,"CANAÃ DOS CARAJÁS, PA, Brazil","PORTO NACIONAL, TO, Brazil","FIGUEIRÓPOLIS, TO, Brazil","PORANGATU, GO, Brazil","ANÁPOLIS, GO, Brazil","IPAMERI,GO, Brazil","RIO VERDE, GO, Brazil","SÃO SIMÃO, GO, Brazil","SÃO JOSÉ DA LAPA, MG, Brazil","LONDRINA, PR, Brazil","MARINGÁ, PR, Brazil"
Origem,,,,,,,,,,,
"Rio Verde, GO, Brazil",87461895.09,53906401.95,43057417.65,33624621.84,15562031.31,19615031.16,0.00,9312480.21,47425454.40,41070891.00,41286667.53
"Jataí, GO, Brazil",84536695.68,58760978.10,47911993.80,38390702.70,20328112.17,24405650.07,4879641.90,10271363.88,52216073.31,41262762.72,41478592.02
"Cristalina, GO, Brazil",83326996.20,46449789.87,35851251.99,27887361.90,12183906.99,8998129.32,26191334.10,28933791.00,31102585.23,54418270.95,53648514.96
"Montividiu, GO, Brazil",87086014.38,53530521.24,42681536.94,33072647.64,17903700.06,22151843.37,2619186.18,12174513.93,49962266.61,43932924.72,44148701.25
"Paraúna, GO, Brazil",80768337.21,47212791.30,36363807.00,26754917.70,11585970.12,16633420.62,6702898.17,14316923.16,50380152.24,45853752.72,48496263.24


In [67]:
df_emission_origem_porto = pd.read_excel('../dados/ibge_emission_origem_porto.xlsx', index_col=0)
df_emission_origem_porto.head()

,"SANTOS, SP, Brazil","VITÓRIA, ES, Brazil","SÃO LUIZ, MA, Brazil","ILHÉUS, BA, Brazil","SALVADOR, BA, Brazil","RIO GRANDE, RS, Brazil","SÃO FRANCISCO DO SUL, SC, Brazil","BARCARENA, PA, Brazil","SANTAREM, PA, Brazil","ITACOATIARA, AM, Brazil","PARANAGUA, PR, Brazil","IMBITUBA, SC, Brazil"
Origem,,,,,,,,,,,,
"Rio Verde, GO, Brazil",52956383.64,73010091.81,1.195554e+08,92055048.66,9.898000e+07,1.123150e+08,70666259.49,1.134420e+08,1.301915e+08,1.749501e+08,66633153.93,81160523.85
"Jataí, GO, Brazil",55953983.49,77800763.49,1.244100e+08,96821182.29,1.037461e+08,1.125069e+08,70858131.21,1.182966e+08,1.255455e+08,1.703041e+08,66825025.65,81352395.57
"Cristalina, GO, Brazil",50321049.84,58768049.28,1.119526e+08,67768183.86,8.132902e+07,1.165825e+08,74933769.39,1.091394e+08,1.318942e+08,1.980027e+08,69902783.13,85428033.75
"Montividiu, GO, Brazil",55493143.08,75546904.02,1.191795e+08,94396717.41,1.013217e+08,1.151770e+08,73528293.21,1.130662e+08,1.276570e+08,1.749370e+08,69495187.65,84022557.57
"Paraúna, GO, Brazil",55911028.71,75964789.65,1.128618e+08,88078987.47,9.500399e+07,1.221725e+08,80523801.03,1.067485e+08,1.287273e+08,1.810132e+08,75492762.00,91018065.39


In [68]:
df_emission_trans_port = pd.read_excel('../dados/ibge_emission_transbordo_porto.xlsx', index_col=0)
df_emission_trans_port.head()

,"SANTOS, SP, Brazil","VITÓRIA, ES, Brazil","SÃO LUIZ, MA, Brazil","ILHÉUS, BA, Brazil","SALVADOR, BA, Brazil","RIO GRANDE, RS, Brazil","SÃO FRANCISCO DO SUL, SC, Brazil","BARCARENA, PA, Brazil","SANTAREM, PA, Brazil","ITACOATIARA, AM, Brazil","PARANAGUA, PR, Brazil","IMBITUBA, SC, Brazil"
"CANAÃ DOS CARAJÁS, PA, Brazil",1.000000e+08,1.000000e+08,17082.52,1.000000e+08,100000000.0,100000000.0,100000000.0,100000000,100000000,100000000,100000000,100000000
"PORTO NACIONAL, TO, Brazil",4.153317e+04,4.704492e+04,22484.95,3.220120e+04,100000000.0,100000000.0,100000000.0,100000000,100000000,100000000,100000000,100000000
"FIGUEIRÓPOLIS, TO, Brazil",3.689432e+04,4.230142e+04,27123.80,2.756235e+04,100000000.0,100000000.0,100000000.0,100000000,100000000,100000000,100000000,100000000
"PORANGATU, GO, Brazil",3.415072e+04,3.955782e+04,29867.40,3.030595e+04,100000000.0,100000000.0,100000000.0,100000000,100000000,100000000,100000000,100000000
"ANÁPOLIS, GO, Brazil",2.781517e+04,3.159777e+04,37932.10,3.826600e+04,100000000.0,100000000.0,100000000.0,100000000,100000000,100000000,100000000,100000000


In [69]:
emission = {}
for n in N:
    for m in M:
        assert((n, m) not in emission)
        emission[n, m] = df_emission_origem_porto[m][n]
    for k in K:
        assert((n, k) not in emission)
        emission[n, k] = df_emission_origem_trans[k][n]
for k in K:
    for m in set(df_emission_trans_port.columns):
        assert((k, m) not in emission)
        emission[k, m] = df_emission_trans_port[m][k]
for i in M:
    emission[i, O[0]] = 0

## Construcao do modelo

In [70]:
m = gp.Model("lalalala")

In [71]:
X = {}
for i in N:
    for j in M:
        X[i, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="supply_{}_port_{}".format(i, j))

for i in N:
    for k in K:
        X[i, k] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="supply_{}_tranship_{}".format(i, k))
                
for j in M:
    for k in K:
        X[k, j] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="tranship_{}_port_{}".format(k, j))

for j in M:
    for o in O:
        X[j, o] = m.addVar(vtype=gp.GRB.CONTINUOUS, lb=0, name="X_{}_{}".format(j, o))

In [72]:
m.setObjectiveN(
    gp.quicksum((X[i, j] * cost[i, j]) for i in N for j in M) + \
    gp.quicksum((X[i, k] * cost[i, k]) for i in N for k in K) + \
    gp.quicksum((X[k, j] * cost[k, j]) for j in M for k in K),
    0,
    priority=2,
    name="Funcao Frete"
)

In [73]:
m.setObjectiveN(
    gp.quicksum((X[i, j] * emission[i, j]) for i in N for j in M) + \
    gp.quicksum((X[i, k] * emission[i, k]) for i in N for k in K) + \
    gp.quicksum((X[k, j] * emission[k, j]) for j in M for k in K),
    1,
    priority=1,
    name="Funcao Emissao CO2"
)

In [75]:
if oferta_total <= demanda_total:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) == supply[i.lower()]
        )
else:
    for i in N:
        m.addConstr(
            (gp.quicksum(X[i, j] for j in M) + gp.quicksum(X[i, k] for k in K)) <= supply[i.lower()]
        )

In [76]:
if oferta_total < demanda_total:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) <= demand[o]
        )
else:
    for o in O:
        m.addConstr(
            gp.quicksum(X[j, o] for j in M) == demand[o]
        )

In [77]:
for j in M:
    m.addConstr(
        (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K)) <= cap_port[j.lower()]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) <= cap_trans[k.lower()]
    )

for k in K:
    m.addConstr(
        gp.quicksum(X[i, k] for i in N) == gp.quicksum(X[k, j] for j in M)
    )

for j in M:
    m.addConstr(
        gp.quicksum(X[j, o] for o in O) == (gp.quicksum(X[i, j] for i in N) + gp.quicksum(X[k, j] for k in K))
    )

In [78]:
rest = m.addConstrs(
    gp.quicksum(X[i, j] for i in N) >= 0 for j in M
)

rest = m.addConstrs(
    gp.quicksum(X[i, k] for i in N) >= 0 for k in K
)

rest = m.addConstrs(
    gp.quicksum(X[k, j] for k in K) >= 0 for j in M
)

In [79]:
m.Params.Method = -1

In [80]:
vals = []

In [81]:
m.setParam('TimeLimit', 60*9)
m.setParam(gp.GRB.Param.Threads, 1)

Set parameter TimeLimit to value 540
Set parameter Threads to value 1


In [82]:
m.update()
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads

Optimize a model with 96 rows, 700 columns and 2786 nonzeros
Model fingerprint: 0x798e099e
Variable types: 700 continuous, 0 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+08]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+04, 2e+08]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 2 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve removed 29 rows and 7 columns
Presolved: 67 rows and 693 columns
------------------------------------------------------------

In [83]:
m.write('out.sol')

In [84]:
X[('Paraúna, GO, Brazil', 'VITÓRIA, ES, Brazil')].x

0.0

In [85]:
df_ans = pd.DataFrame()
writer = pd.ExcelWriter('../dados/ibge_origem_transbordo_result.xlsx', engine='xlsxwriter')

In [86]:
df_ans['Origem'] = N

In [87]:
X[(i, j)].x

0.0

In [99]:
s = 0
for j in M:
    aux = []
    for i in N:
        aux.append(X[(i, j)].x)
    s += sum(aux)
    # df_ans[j] = aux

In [89]:
df_ans

,Origem,"RIO GRANDE, RS, Brazil","SANTOS, SP, Brazil","SÃO FRANCISCO DO SUL, SC, Brazil","VITÓRIA, ES, Brazil","SÃO LUIZ, MA, Brazil","ILHÉUS, BA, Brazil","SALVADOR, BA, Brazil"
0,"São João d'Aliança, GO, Brazil",0.0,0.0000,0.000,0.000,0.0,124600.000,0.000
1,"Montes Claros de Goiás, GO, Brazil",0.0,0.0000,0.000,0.000,0.0,0.000,0.000
2,"Caldas Novas, GO, Brazil",0.0,106740.0000,0.000,0.000,0.0,0.000,0.000
3,"Niquelândia, GO, Brazil",0.0,0.0000,0.000,0.000,0.0,0.000,164300.000
4,"Quirinópolis, GO, Brazil",0.0,0.0000,56442.752,0.000,0.0,0.000,0.000
5,"Ipameri, GO, Brazil",0.0,333080.0000,0.000,0.000,0.0,0.000,0.000
6,"Uruaçu, GO, Brazil",0.0,0.0000,0.000,0.000,0.0,0.000,0.000
7,"Rio Verde, GO, Brazil",0.0,0.0000,0.000,0.000,0.0,0.000,0.000
8,"Doverlândia, GO, Brazil",0.0,0.0000,0.000,0.000,0.0,0.000,0.000
9,"Goiatuba, GO, Brazil",0.0,257122.0000,0.000,0.000,0.0,0.000,0.000


In [90]:
df_ans.to_excel('../dados/ibge_origem_transbordo_result.xlsx')

In [91]:
df_ans = pd.DataFrame()

In [92]:
df_ans['Transbordo'] = K

In [93]:
for j in M:
    aux = []
    for k in K:
        aux.append(X[(k, j)].x)
    s += sum(aux)
    df_ans[j] = aux

In [94]:
df_ans.to_excel('../dados/ibge_transbordo_porto_result.xlsx')

In [100]:
for i in N:
    aux = []
    for k in K:
        aux.append(X[(i, k)].x)
    s += sum(aux)

In [107]:
print(demanda_total, s, oferta_total)

155677582.144 10206211.399999999 10206211.400000002
